In [144]:
# load in libraries

import pandas as pd
import pandas_gbq
from google.cloud import bigquery
from google.oauth2 import service_account

In [135]:
# import csv file to filter sources by country 

country_filter = pd.read_csv('/Users/KevinLubin/Desktop/ds/pearl/gdelt_country_sources.csv')
country_filter.head()

,source,fips
0,0-100.it,IT
1,0-50.ru,RS
2,0-60mag.com,US
3,0-debt.com,US
4,000fff.org,US


In [64]:
# define credentials object for GCP to run queries

credentials = service_account.Credentials.from_service_account_file(
    '/Users/KevinLubin/Desktop/ds/pearl/pearl-336700-0fa91569420d.json')

In [ ]:
WHERE (SQLDATE >= 20210101 AND SQLDATE <= 20210101)

In [205]:
# Perform query

query = """

    SELECT GlobalEventID, SQLDATE, EventCode, EventBaseCode, EventRootCode, Quadclass, AvgTone,
    GoldSteinScale, NumMentions, Sourceurl

    FROM `gdelt-bq.full.events`
    
    WHERE (SQLDATE >= 20210101 AND SQLDATE <= 20210105) 
    
    AND (ActionGeo_CountryCode = 'HA' OR Actor1Geo_CountryCode = 'HA' OR Actor2Geo_CountryCode = 'HA')
    
    GROUP BY SQLDATE, GlobalEventID, EventCode, EventBaseCode, EventRootCode, Quadclass, AvgTone, 
    GoldSteinScale, NumMentions, Sourceurl
    
    ORDER BY SQLDATE, GlobalEventID, EventCode, EventBaseCode, EventRootCode, Quadclass, AvgTone, 
    GoldSteinScale, NumMentions, Sourceurl
    
"""

news_df = pandas_gbq.read_gbq(query, credentials=credentials)

Downloading: 100%|██████████| 376/376 [00:00<00:00, 1215.34rows/s]


In [206]:
news_df.head()

,GlobalEventID,SQLDATE,EventCode,EventBaseCode,EventRootCode,Quadclass,AvgTone,GoldSteinScale,NumMentions,Sourceurl
0,962220767,20210101,173,173,17,4,-11.587983,-5.0,4,http://www.tribune242.com/news/2020/dec/31/45-...
1,962220768,20210101,042,042,04,1,-11.587983,1.9,4,http://www.tribune242.com/news/2020/dec/31/45-...
2,962220769,20210101,173,173,17,4,-11.587983,-5.0,4,http://www.tribune242.com/news/2020/dec/31/45-...
3,962220770,20210101,042,042,04,1,-11.587983,1.9,4,http://www.tribune242.com/news/2020/dec/31/45-...
4,962220931,20210101,043,043,04,1,-11.587983,2.8,4,http://www.tribune242.com/news/2020/dec/31/45-...


In [220]:
# function to look for the base URL for country specific sources and return filtered df

def get_articles(fips):
    
    # find all sources from chosen country
    temp_sources = country_filter[country_filter.fips == fips]
    
    # create a pattern to search for sources in query result
    source_list = temp_sources.source.to_list()
    pattern = '|'.join(source_list)
    
    # create df with results from chosen country
    articles = news_df[news_df.Sourceurl.str.contains(pattern) == True]
    
    # keep unique articles only
    articles = articles.drop_duplicates(subset=['Sourceurl'])
    
    return(articles)

In [221]:
canada_articles = get_articles('CA')

In [224]:
canada_articles.head()

,GlobalEventID,SQLDATE,EventCode,EventBaseCode,EventRootCode,Quadclass,AvgTone,GoldSteinScale,NumMentions,Sourceurl
68,962310574,20210101,010,010,01,1,3.490667,0.0,16,https://www.newswire.ca/news-releases/statemen...
69,962310575,20210101,046,046,04,1,3.490667,7.0,4,https://www.newswire.ca/news-releases/statemen...
70,962310576,20210101,046,046,04,1,3.490667,7.0,12,https://www.newswire.ca/news-releases/statemen...
71,962310577,20210101,070,070,07,2,3.490667,7.0,4,https://www.newswire.ca/news-releases/statemen...
72,962310578,20210101,070,070,07,2,3.490667,7.0,16,https://www.newswire.ca/news-releases/statemen...


In [ ]:
## TODO: Pull in newspaper code to add the text to df. then output for modeling 